In [3]:
pip install yake

You should consider upgrading via the 'c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [9]:
import yake
kw_extractor = yake.KeywordExtractor()
text = """The apps feel lights and fluid however the extra convenience fee does not make sense at all. Way too expensive the more seats you want to order. Very few promotions and the loyalty feature that still work in progress. Hope there are no ads that cover the entire screen for the next update. Could not recommend to everyone for now."""
language = "en"
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 5
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('apps feel lights', 0.001907487553471017)
('extra convenience fee', 0.001907487553471017)
('apps feel', 0.015182204378765869)
('feel lights', 0.015182204378765869)
('lights and fluid', 0.015182204378765869)


In [18]:
max_ngram_size = 3
deduplication_threshold = 0.5
numOfKeywords = 5
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text)
for kw in keywords:
    print(kw)

('apps feel lights', 0.001907487553471017)
('extra convenience fee', 0.001907487553471017)
('lights and fluid', 0.015182204378765869)
('make sense', 0.015182204378765869)
('apps', 0.12229125971027831)


In [20]:
text2="""Very easy to use and more promotion can be found. Preview trailer of movies are available too."""
keywords2 = custom_kw_extractor.extract_keywords(text2)
for kw in keywords2:
    print(kw)

('found', 0.2005079697193566)
('easy', 0.36073110501666333)
('promotion', 0.36073110501666333)
('Preview trailer', 0.4456055016437946)
('Preview', 0.46553351027698087)


In [22]:
for kw in keywords2:
    print(kw.type())

AttributeError: 'tuple' object has no attribute 'type'

In [ ]:
import nltk
nltk.download('punkt')
import re
import pprint
from nltk import Tree

In [1]:
import nltk
#nltk.download('punkt')
import re
import pprint
from nltk import Tree

grammar = """
    NP: {<JJ>*<NN*>+}
    {<JJ>*<NN*><CC>*<NN*>+}
    {<DT>*<JJ>*<NN>+}
    {<DT>*<JJ>*<NNS>+}
    {<JJ>*<NNS>+}
    """

NPchunked = nltk.RegexpParser(grammar)

def chunked_text(text):
    sentences = nltk.sent_tokenize(text)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    sentences = [NPchunked.parse(sent) for sent in sentences]
    return sentences

def NP_chunked(sentences):
    nps = []
    NPchunked = nltk.RegexpParser(grammar)
    for sent in sentences:
        tree = NPchunked.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP':
                t = subtree
                t = ' '.join(word for word, tag in t.leaves())
                nps.append(t)
    return nps

para = "Today is a very great day. Indian politicians are very corrupt."
sentences = chunked_text(para)
np =NP_chunked(sentences)
print(np)

['Today', 'great day', 'Indian politicians']


In [8]:
para = "The application run well, but the UI design is so bad"
sentences = chunked_text(para)
np =NP_chunked(sentences)
print(sentences)
print(np)

[Tree('S', [('The', 'DT'), Tree('NP', [('application', 'NN'), ('run', 'NN')]), ('well', 'RB'), (',', ','), ('but', 'CC'), ('the', 'DT'), ('UI', 'NNP'), Tree('NP', [('design', 'NN')]), ('is', 'VBZ'), ('so', 'RB'), ('bad', 'JJ')])]
['application run', 'design']


In [2]:
import pandas as pd
data = {'Text':["❤️", "All in one. You got meditation, music, even yoga and journaling❤️❤️", "Nice", "The app takes so long to load :()","😠😠"]}
df=pd.DataFrame(data)
def post(text):
    return NP_chunked(chunked_text(text))

df['chunked'] = df.apply(lambda x: post(x.Text),axis=1)
df

,Text,chunked
0,❤️,[❤️]
1,"All in one. You got meditation, music, even yo...","[meditation, music, journaling❤️❤️]"
2,Nice,[Nice]
3,The app takes so long to load :(),"[app, load]"
4,😠😠,[😠😠]


In [6]:
def post(text):
    return NP_chunked(chunked_text(text))

df['chunked'] = df.apply(lambda x: post(x.Text),axis=1)
df

,Text,chunked
0,"The application run well, but the user interfa...","[application run, user interface design]"
1,"All in one. You got meditation, music, even yo...","[meditation, music, journaling]"
2,Nice,[Nice]
3,The app takes so long to load,[app]


In [9]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
df['vader_score'] = df['Text'].apply(lambda Text: sid.polarity_scores(Text))
df['compound']  = df['vader_score'].apply(lambda score_dict: score_dict['compound'])
df['positive']  = df['vader_score'].apply(lambda score_dict: score_dict['pos'])
df['negative']  = df['vader_score'].apply(lambda score_dict: score_dict['neg'])

def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

df['rating'] = df.apply(lambda x: getrating(x.compound,x.positive,x.negative),axis=1)

df.head()

,Text,chunked,vader_score,compound,positive,negative,rating
0,"The application run well, but the user interfa...","[application run, user interface design]","{'neg': 0.351, 'neu': 0.562, 'pos': 0.087, 'co...",-0.7709,0.087,0.351,-1
1,"All in one. You got meditation, music, even yo...","[meditation, music, journaling]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.000,0.000,0
2,Nice,[Nice],"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4215,1.000,0.000,2
3,The app takes so long to load,[app],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.000,0.000,0


In [18]:
emo = {'Text':["❤️", "All in one. You got meditation, music, even yoga and journaling❤️❤️", "Nice", "The app takes so long to load :()","😠😠"]}
emoji=pd.DataFrame(emo)

emoji['vader_score'] = emoji['Text'].apply(lambda Text: sid.polarity_scores(Text))
emoji['compound']  = emoji['vader_score'].apply(lambda score_dict: score_dict['compound'])
emoji['positive']  = emoji['vader_score'].apply(lambda score_dict: score_dict['pos'])
emoji['negative']  = emoji['vader_score'].apply(lambda score_dict: score_dict['neg'])
emoji['rating'] = emoji.apply(lambda x: getrating(x.compound,x.positive,x.negative),axis=1)

emoji

,Text,vader_score,compound,positive,negative,rating
0,❤️,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,0.0,0
1,"All in one. You got meditation, music, even yo...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,0.0,0
2,Nice,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4215,1.0,0.0,2
3,The app takes so long to load :(),"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,0.0,0
4,😠😠,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.0,0.0,0


In [19]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
dataemo = ["❤️", "All in one. You got meditation, music, even yoga and journaling❤️❤️", "Nice", "The app takes so long to load :()","😠😠"]

In [10]:
dataemo2 = ["aplikasinya jelek","aplikasinya bagus, ada fitur yang membantu"]

In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

NameError: name 'dataemo' is not defined

In [4]:
for sentence in dataemo:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(vs)))

❤️-------------------------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
All in one. You got meditation, music, even yoga and journaling❤️❤️ {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Nice------------------------------------ {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
The app takes so long to load :()------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
😠😠-------------------------------------- {'neg': 0.767, 'neu': 0.233, 'pos': 0.0, 'compound': -0.765}


In [12]:
for sentence in dataemo2:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(vs)))

aplikasinya jelek----------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
aplikasinya bagus, ada fitur yang membantu {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [16]:
!pip uninstall vaderSentiment

^C


In [ ]:
!pip install vader-multi

In [24]:
Text = ["The application run well, but the user interface design is so bad", "All in one. You got meditation, music, even yoga and journaling", "Nice", "The app takes so long to load","The part was savage","The party was bomb"]

for sentence in Text:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(vs)))

The application run well, but the user interface design is so bad {'neg': 0.351, 'neu': 0.562, 'pos': 0.087, 'compound': -0.7709}
All in one. You got meditation, music, even yoga and journaling {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Nice------------------------------------ {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
The app takes so long to load----------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
The part was savage--------------------- {'neg': 0.5, 'neu': 0.5, 'pos': 0.0, 'compound': -0.4588}
The party was bomb---------------------- {'neg': 0.405, 'neu': 0.253, 'pos': 0.342, 'compound': -0.128}


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
aa = analyzer.polarity_scores(data)
aa

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [15]:
tes = pd.DataFrame([aa])
tes

,neg,neu,pos,compound
0,0.351,0.562,0.087,-0.7709


In [ ]:
filtered['compound']  = filtered['vader_score'].apply(lambda score_dict: score_dict['compound'])
filtered['positive']  = filtered['vader_score'].apply(lambda score_dict: score_dict['pos'])
filtered['negative']  = filtered['vader_score'].apply(lambda score_dict: score_dict['neg'])

In [23]:
#the data
data = {'Text':["❤️❤️❤️", "💘💘",":)","😁","😁😁","Got everything i need","All in one. You got meditation, music, even yoga and journaling❤️❤️", "Nice", "The app takes so long to load :()","😠😠"]}
df=pd.DataFrame(data)


#chunking
def post(text):
    return NP_chunked(chunked_text(text))

df['chunked'] = df.apply(lambda x: post(x.Text),axis=1)


#vader
df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['Text']]
df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['Text']]
df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['Text']]
df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['Text']]


#convert rating
def getrating (compound,positive,negative):
    if compound > 0.001:
        if positive > 0.5:
            return 2
        else:
            return 1
    elif compound < -0.001:
        if negative > 0.5:
            return -2
        else:
            return -1
    elif (compound > -0.001) and (compound < 0.001):
        return 0

df['rating'] = df.apply(lambda x: getrating(x.compound,x.pos,x.neg),axis=1)

df

,Text,chunked,compound,neg,neu,pos,rating
0,❤️❤️❤️,[❤️❤️❤️],0.0000,0.000,1.000,0.000,0
1,💘💘,[💘💘],0.8555,0.000,0.323,0.677,2
2,:),[],0.4588,0.000,0.000,1.000,2
3,😁,[😁],0.4588,0.000,0.571,0.429,1
4,😁😁,[😁😁],0.7184,0.000,0.571,0.429,1
5,Got everything i need,"[everything, i]",0.0000,0.000,1.000,0.000,0
6,"All in one. You got meditation, music, even yo...","[meditation, music, journaling❤️❤️]",0.0000,0.000,1.000,0.000,0
7,Nice,[Nice],0.4215,0.000,0.000,1.000,2
8,The app takes so long to load :(),"[app, load]",0.0000,0.000,1.000,0.000,0
9,😠😠,[😠😠],-0.7650,0.767,0.233,0.000,-2


In [24]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: textblob in c:\users\asus\appdata\local\programs\python\python39\lib\site-packages (0.17.1)



You should consider upgrading via the 'c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [25]:
from textblob import TextBlob

TextBlob('coool apps!').correct()

TextBlob("cool apes!")

In [32]:
TextBlob('didnt work').correct()

TextBlob("didn work")

In [38]:
NP_chunked(chunked_text("i am having trouble with the meditation"))

['i', 'trouble', 'meditation']

In [ ]:
ulasan_c =ulasan_c.replace("won't","will not")
ulasan_c =ulasan_c.replace("app ","application ")
ulasan_c =ulasan_c.replace("n't"," not")


In [35]:
text="didnt"
text.replace("did not")

'wo not work'

In [41]:
!pip install autocorrect
from autocorrect import Speller
spell = Speller(lang='en')

You should consider upgrading via the 'c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622380 sha256=8a3a26f792db5834a91c89f530a06e8db03d978eb227e39cff62a5dfdba1e83d
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\ab\0f\23\3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect


In [42]:
def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

remove_html("0827 nomer wa ku")

'0827 nomer wa ku'

In [45]:
import string
from nltk.tokenize import word_tokenize
s = set(string.punctuation)
sentence = "hi semua! apa kabar???? 123 cek haloo,,,???"
sentence = word_tokenize(sentence)
filtered_word = []
for i in sentence:
    if i not in s:
        filtered_word.append(i)
for word in filtered_word:
    print(word,end=" ")

hi semua apa kabar 123 cek haloo 

In [ ]:
from autocorrect import Speller
spell = Speller(lang='en')